In [2]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
import io

# Nivel Pais

In [246]:
global_biweekly_cases = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/biweekly_cases_per_million.csv'
r = requests.get(global_biweekly_cases)
rawData = pd.read_csv(io.StringIO(r.content.decode('utf-8')))
raw_long= pd.melt(rawData, id_vars=['date'], value_vars=rawData.columns[1:])
raw_long.rename(columns={'variable':'location', 'value':'biweekly_million'}, inplace=True)
raw_long['nicv'] = raw_long['biweekly_million']*1000/1000000
global_d = raw_long[['date', 'location', 'nicv']]

In [336]:
global_d['country']=global_d['location']
global_d.date=pd.to_datetime(global_d['date'], format = '%Y-%m-%d')

In [337]:
global_d

,date,location,nicv,country
0,2020-01-22,World,NaN,World
1,2020-01-23,World,NaN,World
2,2020-01-24,World,NaN,World
3,2020-01-25,World,NaN,World
4,2020-01-26,World,NaN,World
...,...,...,...,...
86759,2021-03-28,Zimbabwe,0.022741,Zimbabwe
86760,2021-03-29,Zimbabwe,0.022539,Zimbabwe
86761,2021-03-30,Zimbabwe,0.020454,Zimbabwe
86762,2021-03-31,Zimbabwe,0.022203,Zimbabwe


In [248]:
def get_nicv_country(country, last_day = 1):
    '''
    '''
    val = raw_long[(raw_long['location'] == country) & 
                   (raw_long['date']== str(dt.datetime.today() - 
                                           dt.timedelta(days=last_day)).split()[0])].nicv.values
    
    if len(val) == 0:
        return None
    
    return val[0]

In [249]:
get_nicv_country('Brazil')

4.982249

# Nivel County/Municipio

## United States

In [250]:
start = dt.datetime.strptime('04-12-2020', '%m-%d-%Y') 
end = dt.datetime.today()
step = dt.timedelta(days=1)
dates = []
while start < end:
    dates.append(start.strftime('%m-%d-%Y'))
    start += step
    str(start)

In [251]:
url_county_us = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv"

In [252]:
bigraw = None 
for i, date in enumerate(dates[:-1]):
    r = requests.get(url_county_us.format(date))
    rawData = pd.read_csv(io.StringIO(r.content.decode('utf-8')))
    rawData = rawData[rawData.Country_Region=='US'][['FIPS', 'Admin2', 
                                                'Province_State', 'Country_Region', 'Combined_Key',
                                                'Lat', 'Long_', 'Confirmed']]
    rawData['date'] = date
    if i == 0:
        bigraw = rawData
    else:
        bigraw = bigraw.append(rawData)

In [253]:
bigraw= bigraw[bigraw.Admin2 != 'Unassigned']

In [254]:
r = requests.get("https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=county:*")
populations = pd.DataFrame(r.json()[1:], columns=r.json()[0])
populations['FIPS'] = (populations['state']+ populations['county']).astype(float).astype(str)
populations['POP'] = populations['POP'].astype(float)

In [255]:
bigraw['FIPS'] = bigraw.groupby(['Combined_Key'])['FIPS'].transform(lambda x: 
                                                                    x.fillna(x.astype(float).min())).astype(str)

In [291]:
big = bigraw.merge(populations)
big['date'] = pd.to_datetime(big['date'])
big = big.sort_values(by=['Combined_Key', 'date']).reset_index(drop=True)

In [292]:
vals = big.groupby(['Combined_Key'])\
                    .apply(lambda x:
                           x.set_index('date')).rolling(15)['Confirmed'].apply(lambda x:x[-1] - x[0],
                                                                               raw=True).reset_index()

In [293]:
vals.index = big.index
big['last14'] = vals['Confirmed']
big['nicv']= big['last14']/big['POP'].astype(float)*1000

In [294]:
big[(big.date =='2021-03-26') & (big.Admin2=='Cook')][['Combined_Key', 'nicv']]

,Combined_Key,nicv
224373,"Cook, Georgia, US",1.158078
224728,"Cook, Illinois, US",2.145534
225021,"Cook, Minnesota, US",2.562694


In [296]:
big.rename(columns={'Combined_Key':'location', 'Province_State':'state_name'}, inplace=True)

In [297]:
us = big[['date', 'location', 'nicv', 'state_name']]
us['country']='United States'

<ipython-input-297-fb6816ea7fac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us['country']='United States'


In [298]:
us

,date,location,nicv,state_name,country
0,2020-04-12,"Abbeville, South Carolina, US",NaN,South Carolina,United States
1,2020-04-13,"Abbeville, South Carolina, US",NaN,South Carolina,United States
2,2020-04-14,"Abbeville, South Carolina, US",NaN,South Carolina,United States
3,2020-04-15,"Abbeville, South Carolina, US",NaN,South Carolina,United States
4,2020-04-16,"Abbeville, South Carolina, US",NaN,South Carolina,United States
...,...,...,...,...,...
1108022,2021-03-28,"Ziebach, South Dakota, US",0.362845,South Dakota,United States
1108023,2021-03-29,"Ziebach, South Dakota, US",0.362845,South Dakota,United States
1108024,2021-03-30,"Ziebach, South Dakota, US",0.362845,South Dakota,United States
1108025,2021-03-31,"Ziebach, South Dakota, US",1.088534,South Dakota,United States


## Mexico Municipios

In [261]:
mex_mun = 'https://datos.covid-19.conacyt.mx/Downloads/Files/Casos_Diarios_Municipio_Confirmados_20210401.csv'

In [262]:
r = requests.get(mex_mun)
mex = pd.read_csv(io.StringIO(r.content.decode('utf-8')))

In [263]:
mex = pd.melt(mex, id_vars=mex.columns[:3], value_vars=mex.columns[3:])
mex.rename(columns={'variable':'date', 'value':'daily'}, inplace=True)

In [264]:
claves_entidades = {1: 'Aguascalientes', 2:'Baja California', 3:'Baja California Sur',
                   4:'Campeche', 5:'Coahuila', 6:'Colima', 7:'Chiapas', 8:'Chihuahua',
                   9: 'Ciudad de México', 10:'Durango', 11:'Guanajuato', 12:'Guerrero',
                   13:"Hidalgo", 14:'Jalisco', 15: 'Estado de México', 16:'Michoacán',
                   17: 'Morelos', 18:'Nayarit', 19:'Nuevo León', 20:'Oaxaca',
                   21:'Puebla', 22:'Querétaro', 23:'Quintana Roo', 24:'San Luis Potosí',
                   25:'Sinaloa', 26:'Sonora', 27:'Tabasco', 28:'Tamaulipas',
                   29:'Tlaxcala', 30:'Veracruz', 31:'Yucatán', 32:'Zacatecas'}

In [265]:
mex['state'] = mex.apply(lambda x: claves_entidades[x['cve_ent']//1000], axis=1)

In [340]:
mex['location'] = mex['nombre'] +", " + mex['state']+', Mexico'

In [267]:
mex[(mex.nombre == 'Puebla') & (mex.date == '24-03-2021')]

,cve_ent,poblacion,nombre,date,daily,state,location
1095295,21114,1698509,Puebla,24-03-2021,101,Puebla,"Puebla, Puebla"


In [268]:
mex['date'] = pd.to_datetime(mex['date'], format = '%d-%m-%Y')

In [269]:
mex = mex.sort_values(by=['cve_ent', 'date']).reset_index(drop=True)

In [270]:
mex['confirmed'] = mex.groupby(['cve_ent']).daily.cumsum()

In [349]:
rolling14 = mex.groupby(['cve_ent'])\
                    .apply(lambda x:
                           x.set_index('date')).rolling(15)['confirmed'].apply(lambda x:x[-1]- x[0],
                                                                               raw=True).reset_index()

In [350]:
rolling14.index = mex.index
mex['last14'] = rolling14['confirmed']
mex['nicv']= mex['last14']*1000/(mex['poblacion'].astype(float))

In [355]:
mex[(mex.location == 'Puebla, Puebla, Mexico') & (mex.date == '2021-03-20')]

,cve_ent,poblacion,nombre,date,daily,state,location,confirmed,last14,nicv
761934,21114,1698509,Puebla,2021-03-20,81,Puebla,"Puebla, Puebla, Mexico",49143,1713.0,1.008532


In [361]:
puebla = mex[(mex.location == 'Puebla, Puebla, Mexico')]
puebla[puebla.nicv == puebla.nicv.max()]

,cve_ent,poblacion,nombre,date,daily,state,location,confirmed,last14,nicv
761874,21114,1698509,Puebla,2021-01-19,412,Puebla,"Puebla, Puebla, Mexico",38233,4932.0,2.903723


In [365]:
mex[(mex.nombre == 'San Andres Cholula') & (mex.date == '2020-12-21')]

,cve_ent,poblacion,nombre,date,daily,state,location,confirmed,last14,nicv
764110,21119,154192,San Andres Cholula,2020-12-21,6,Puebla,"San Andres Cholula, Puebla, Mexico",1408,121.0,0.784736


In [341]:
mex_d = mex[['date', 'location', 'state']]
mex_d.rename(columns={'state':'state_name'}, inplace=True)
mex_d['country']= 'Mexico'

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-341-9920f174c4dc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mex_d['country']= 'Mexico'


In [342]:
mex_d

,date,location,state_name,country
0,2020-01-04,"Aguascalientes, Aguascalientes, Mexico",Aguascalientes,Mexico
1,2020-01-05,"Aguascalientes, Aguascalientes, Mexico",Aguascalientes,Mexico
2,2020-01-06,"Aguascalientes, Aguascalientes, Mexico",Aguascalientes,Mexico
3,2020-01-07,"Aguascalientes, Aguascalientes, Mexico",Aguascalientes,Mexico
4,2020-01-08,"Aguascalientes, Aguascalientes, Mexico",Aguascalientes,Mexico
...,...,...,...,...
1113016,2021-03-27,"Santa Maria de la Paz, Zacatecas, Mexico",Zacatecas,Mexico
1113017,2021-03-28,"Santa Maria de la Paz, Zacatecas, Mexico",Zacatecas,Mexico
1113018,2021-03-29,"Santa Maria de la Paz, Zacatecas, Mexico",Zacatecas,Mexico
1113019,2021-03-30,"Santa Maria de la Paz, Zacatecas, Mexico",Zacatecas,Mexico


# Merge All

In [343]:
all_mun = global_d.append(us).append(mex_d)

In [344]:
all_mun

,date,location,nicv,country,state_name
0,2020-01-22,World,NaN,World,NaN
1,2020-01-23,World,NaN,World,NaN
2,2020-01-24,World,NaN,World,NaN
3,2020-01-25,World,NaN,World,NaN
4,2020-01-26,World,NaN,World,NaN
...,...,...,...,...,...
1113016,2021-03-27,"Santa Maria de la Paz, Zacatecas, Mexico",NaN,Mexico,Zacatecas
1113017,2021-03-28,"Santa Maria de la Paz, Zacatecas, Mexico",NaN,Mexico,Zacatecas
1113018,2021-03-29,"Santa Maria de la Paz, Zacatecas, Mexico",NaN,Mexico,Zacatecas
1113019,2021-03-30,"Santa Maria de la Paz, Zacatecas, Mexico",NaN,Mexico,Zacatecas


In [367]:
all_mun.to_csv('nicv-01.csv', index=False)